In [2]:
import json
import numpy as np
import os
import pandas as pd
import pathlib

In [3]:
JSON_DIR_PATH = 'data/output/json'
PAID_DATE = 'paid_date'
CATEGORY = 'income'
BASE_DIR = '/Users/dev-yoshiki/Hobby/paycheck_reader'
EMP_NO = 9999999
JSON_DIR_PATH = '/Users/dev-yoshiki/Hobby/paycheck_reader/data/output/json'
JSON_FILES = ['1300_1614120_K_20180425.json', '1300_1614120_K_20180525.json', '1300_1614120_K_20180625.json', '1300_1614120_K_20180725.json', '1300_1614120_K_20180824.json', '1300_1614120_K_20180925.json', '1300_1614120_K_20181025.json', '1300_1614120_K_20181122.json', '1300_1614120_K_20181225.json', '1300_1614120_K_20190125.json', '1300_1614120_K_20190225.json', '1300_1614120_K_20190325.json', '1300_1614120_K_20190425.json', '1300_1614120_K_20190524.json', '1300_1614120_K_20190625.json', '1300_1614120_K_20190725.json', '1300_1614120_K_20190823.json', '1300_1614120_K_20190925.json', '1300_1614120_K_20191025.json', '1300_1614120_K_20191125.json', '1300_1614120_K_20191225.json', '1300_1614120_K_20200124.json', '1300_1614120_K_20200225.json']

In [4]:
dataframes = []
for filename in JSON_FILES:
    dates, keys, values, indexes = [], [], [], []

    file_path = pathlib.Path(JSON_DIR_PATH, filename)
    with open(file_path, 'r') as json_file:
        dict_data = json.load(json_file)

    """Single key extraction"""
    dates, keys, values = [], [], []
    date = dict_data[PAID_DATE]
    for key, value in dict_data['incomes'].items():
        values.append(value)
        keys.append(key)
        dates.append(date)
    df = pd.DataFrame({'date': dates, 'type': keys, 'value': values})
    dataframes.append(df)

df = pd.concat(dataframes)

In [37]:
# Sort data by ranking values statistically.
table = pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)
table.loc['不足日数時間精算'] = table.loc['不足日数時間精算'].abs()
table.replace(0, np.nan, inplace=True)

## total, mean, std
table['total'] = table.filter(regex='\d{4}-\d{2}-\d{2}').sum(axis=1)
table['total'] = table['total'].astype(int).round(0)
table['mean'] = table.filter(regex='\d{4}-\d{2}-\d{2}').mean(axis=1)
table['mean'] = table['mean'].astype(int).round(0)
table = table.fillna(0)
table['std'] = table.filter(regex='\d{4}-\d{2}-\d{2}').std(axis=1)
table['std'] = (table['std'] / table['mean'] * 100).astype(float).round(2)
# table['std'] = table['std'].astype(int).round(0)

# norm
Total = table['total'].sum()
table['norm'] = (table['total'] / Total * 100).round(2)
print('Total:', Total)
print('sum(norm):', table['norm'].sum())

# sort by std


table[['total', 'mean', 'std', 'norm']]

Total: 7240934
sum(norm): 100.00999999999999


date,total,mean,std,norm
type,,,,
Smart&Fun!支援金,230000,10000,0.00,3.18
その他支給(課税加算),50000,50000,20.85,0.69
不足日数時間精算,40979,10244,43.17,0.57
休日勤務手当,51585,12896,42.25,0.71
基本給,5100500,221760,1.01,70.44
新卒住宅補助,500000,25000,34.44,6.91
時間外勤務手当,936814,44610,55.57,12.94
深夜勤務手当,7507,834,78.78,0.10
資格取得(奨励金),35700,17850,31.72,0.49


In [30]:
table[['mean','total']]

date,mean,total
type,,
Smart&Fun!支援金,10000,230000
その他支給(課税加算),50000,50000
不足日数時間精算,10244,40979
休日勤務手当,12896,51585
基本給,221760,5100500
新卒住宅補助,25000,500000
時間外勤務手当,44610,936814
深夜勤務手当,834,7507
資格取得(奨励金),17850,35700


In [ ]:
newTable = table.transpose()
EN_TYPE = ['Alfa', 'Bravo', 'Charlie', 'Delta', 'Echo', 'Foxtrot',
        'Golf', 'Hotel', 'India', 'Juliett', 'Kilo', 'LIma', 'Mike', 
        'November', 'Oscar', 'Papa', 'Quebec', 'Romeo', 'Sierra']

col_num = len(table.columns)
EN_TYPE = EN_TYPE[:col_num]
col_dict = dict(zip(table.columns, EN_TYPE))
newTable = table.rename(columns=col_dict)
newTable